<a href="https://colab.research.google.com/github/PratikStar/google-colab/blob/main/1_2_Create_Audio_Spectrogram_(Windowed_version).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Pipeline Classes

In [ ]:
"""
1- load a file..
2- pad the signal (if necessary)
3- extracting log spectrogram from signal
4- normalise spectrogram
5- save the normalised spectrogram

PreprocessingPipeline
"""
import os
import pickle
import re
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import inspect
import soundfile as sf

SHOW_LOGS = True
LOG_LEVEL = 10
def log(logline, log_level = 1):
    if SHOW_LOGS and log_level <= LOG_LEVEL:
        stack = inspect.stack()
        the_class = stack[1][0].f_locals["self"].__class__.__name__
        the_method = stack[1][0].f_code.co_name
        print(the_class + ": " + logline)

class Loader:
    """Loader is responsible for loading an audio file."""

    def __init__(self, sample_rate, offset, load_duration, window_duration, mono):
        self.sample_rate = sample_rate
        self.load_duration = load_duration
        self.window_duration = window_duration
        self.offset = offset
        self.mono = mono

    def load(self, file_path):
        signal = librosa.load(file_path,
                              sr=self.sample_rate,
                              duration=self.load_duration,
                              offset=self.offset,
                              mono=self.mono)[0]
        log("Shape of the loaded signal: " + str(signal.shape), 1)
        log("Mean of the loaded signal: " + str(np.mean(signal)), 5)
        log("Min of the loaded signal: " + str(np.amin(signal)), 5)
        log("Max of the loaded signal: " + str(np.amax(signal)), 5)
        log("Raw signal: " + str(signal), 5)
        return signal


class Padder:
    """Padder is responsible to apply padding to an array."""

    def __init__(self, mode="constant"):
        self.mode = mode

    def left_pad(self, array, num_missing_items):
        padded_array = np.pad(array,
                              (num_missing_items, 0),
                              mode=self.mode)
        return padded_array

    def right_pad(self, array, num_missing_items):
        padded_array = np.pad(array,
                              (0, num_missing_items),
                              mode=self.mode)
        return padded_array


class LogSpectrogramExtractor:
    """LogSpectrogramExtractor extracts log spectrograms (in dB) from a
    time-series signal.
    """

    def __init__(self, frame_size, hop_length):
        self.frame_size = frame_size
        self.hop_length = hop_length

    def extract(self, signal):
        stft = librosa.stft(signal,
                            n_fft=self.frame_size,
                            hop_length=self.hop_length)[:-1]
        log("Shape of stft: " + str(stft.shape), 1)
        spectrogram = np.abs(stft)
        log_spectrogram = librosa.amplitude_to_db(spectrogram)
        return log_spectrogram

class FeatureExtractor:
    """ Extracts the required features"""

    def extract_ampid_from_name(self, filename):
        amp_id = int(filename.split('-')[0])
        log("Amp ID: " + str(amp_id), 5)
        return amp_id
    def extract_clipid_from_name(self, filename):
        clip_id = int(filename.split('-')[0])
        log("Clip ID: " + str(clip_id), 5)
        return clip_id

class MinMaxNormaliser:
    """MinMaxNormaliser applies min max normalisation to an array."""

    def __init__(self, min_val, max_val):
        self.min = min_val
        self.max = max_val

    def normalise(self, array):

        norm_array = (array - array.min()) / (array.max() - array.min())
        norm_array = norm_array * (self.max - self.min) + self.min
        log("Shape of norm_array: " + str(norm_array.shape), 1)
        log("Max of norm_array: " + str((array.max())), 5)
        log("Min of norm_array: " + str((array.min())), 5)
        return norm_array

    def denormalise(self, norm_array, original_min, original_max):
        array = (norm_array - self.min) / (self.max - self.min)
        array = array * (original_max - original_min) + original_min
        return array


class Saver:
    """saver is responsible to save features, and the min max values."""

    def __init__(self, feature_save_dir, min_max_values_save_dir):
        self.feature_save_dir = feature_save_dir
        Utils._create_folder_if_it_doesnt_exist(self.feature_save_dir)
        self.min_max_values_save_dir = min_max_values_save_dir

    def save_feature(self, feature, file_name):
        save_path = self._generate_save_path(file_name)
        np.save(save_path, feature)

    def save_min_max_values(self, min_max_values):
        save_path = os.path.join(self.min_max_values_save_dir,
                                 "min_max_values.pkl")
        self._save(min_max_values, save_path)

    @staticmethod
    def _save(data, save_path):
        with open(save_path, "wb") as f:
            pickle.dump(data, f)

    def _generate_save_path(self, file_name):
        save_path = os.path.join(self.feature_save_dir, file_name + ".npy")
        return save_path

class Visualizer:
    def __init__(self, file_dir, frame_size, hop_length, offset):
        self.file_dir = file_dir
        self.frame_size = frame_size
        self.hop_length = hop_length
        self.offset = offset
        pass
    
    def visualize(self, spectrogram, file_name):
        plt.ioff()
        fig, ax = plt.subplots()
        img = librosa.display.specshow(spectrogram, y_axis='log', x_axis='time', ax=ax)
        ax.set_title("Frame Size: " + str(self.frame_size) + ", Hop Length: " + str(self.hop_length) + ", Offset: " + str(self.offset))
        fig.colorbar(img, ax=ax, format="%+2.0f dB")
        Utils._create_folder_if_it_doesnt_exist(self.file_dir)
        fig.savefig(os.path.join(self.file_dir, file_name + ".png"))
        plt.close(fig)

class Utils:
    def __init__(self):
        pass
    @staticmethod
    def _create_folder_if_it_doesnt_exist(folder):
        if not os.path.exists(folder):
            os.makedirs(folder)

class AudioReconstructor:
    """Processes stft inverse and saves the audio file"""
    def __init__(self, file_dir, hop_length, frame_size):
        self.file_dir = file_dir
        self.hop_length = hop_length
        self.frame_size = frame_size

    def reconstruct(self, features, file_name):
        # Invert using Griffin-Lim
        log("Shape of features: " + str(features.shape), 1)
        features_inv = librosa.griffinlim(features,
                                          hop_length=self.hop_length,
                                          win_length=self.frame_size)
        log("Shape of Inverse features: " + str(features_inv.shape), 1)
        Utils._create_folder_if_it_doesnt_exist(self.file_dir)
        sf.write(os.path.join(self.file_dir, file_name ), y_inv, 22050)
        # save audio
    def reconstruct_from_path(self, file_path):
        # Invert using Griffin-Lim
        features = np.load(file_path)
        self.reconstruct(features, os.path.basename(file_path))
    
class PreprocessingPipeline:
    """PreprocessingPipeline processes a single audio file, applying
    the following steps to each file:
        1- load a file
        2- pad the signal (if necessary)
        3- extracting log spectrogram from signal
        4- normalise spectrogram
        5- save the normalised spectrogram

    Storing the min max values for all the log spectrograms.
    """

    def __init__(self):
        self.padder = None
        self.extractor = None
        self.feature_extractor = None
        self.normaliser = None
        self.saver = None
        self.visualizer = None
        self.reconstructor = None
        self.min_max_values = {}
        self._loader = None
        self._num_expected_samples = None

    @property
    def loader(self):
        return self._loader

    @loader.setter
    def loader(self, loader):
        self._loader = loader
        self._num_expected_samples = int(loader.sample_rate * loader.window_duration)

    def process_file(self, audio_files_dir, audio_file_name, window_id, save_flg=True, visualize_flg=True, external_features_flg=False):

        # self._process_file(root, file, save_flg, visualize_flg, external_features_flg)

        file_path = os.path.join(audio_files_dir, audio_file_name)
        signal = self.loader.load(file_path)
        if external_features_flg:
            external_features = [ self.feature_extractor.extract_ampid_from_name(audio_file_name),
                                  self.feature_extractor.extract_clipid_from_name(audio_file_name),
                                  window_id                         
                                ]
            signal = np.concatenate((external_features, signal))

        if self._is_padding_necessary(signal):
            signal = self._apply_padding(signal)
        feature = self.extractor.extract(signal)

        norm_feature = self.normaliser.normalise(feature)
        print(norm_feature)
        name_split = audio_file_name.split(" ")
        file_name = name_split[0] + "-" + ('%02d' % (window_id)) + " " + " ".join(name_split[1:])
        print("\t" + file_name)
        if visualize_flg:
            self.visualizer.visualize(norm_feature, file_name)

        if save_flg:
            save_path = self.saver.save_feature(norm_feature, file_name)
            # self._store_min_max_value(save_path, feature.min(), feature.max())
        log(f"Processed file {audio_file_name}", 5)

        # self.saver.save_min_max_values(self.min_max_values)

    def _is_padding_necessary(self, signal):
        if len(signal) < self._num_expected_samples:
            log("Padding necessary", 5)
            log("Signal: " + str(len(signal)), 5)
            log("Exp samples: " + str(self._num_expected_samples), 5)
            return True
        return False

    def _apply_padding(self, signal):
        num_missing_samples = self._num_expected_samples - len(signal)
        padded_signal = self.padder.right_pad(signal, num_missing_samples)
        return padded_signal

    def _store_min_max_value(self, save_path, min_val, max_val):
        self.min_max_values[save_path] = {
            "min": min_val,
            "max": max_val
        }


## Driver Code

In [ ]:
## TOBE <AMP_ID>-<CLIP_ID>-<WINDOW_ID> <AMP_NAME>.wav.npy
## 00000-06-00 DI.wav.png is not present

FRAME_SIZE = 512
HOP_LENGTH = FRAME_SIZE // 2 # smaller better?
OFFSET = 0
WINDOW_DURATION = 0.74  # in seconds
SAMPLE_RATE = 22050
MONO = True

# Dimensions: (FRAME_SIZE / 2) x ceil(SAMPLE_RATE * DURATION / HOP_LENGTH)

## SOURCE
AUDIO_FILES_DIR = "/content/drive/MyDrive/Music/VAE/audio/"
AUDIO_REGEX = "00000-06.*\.wav$"

## OUTPUT
SPECTROGRAMS_SAVE_DIR = "/content/drive/MyDrive/Music/VAE/ICASSP/spectrogram/"
SPECTROGRAMS_IMG_SAVE_DIR = "/content/drive/MyDrive/Music/VAE/ICASSP/spectrogram-img/"
RECONSTRUCTED_AUDIO_SAVE_DIR = "/content/drive/MyDrive/Music/VAE/ICASSP/audio-reconstructed/"
MIN_MAX_VALUES_SAVE_DIR = "/content/drive/MyDrive/Music/VAE/ICASSP"

NUMBER_OF_CLIPS = 100000

# instantiate all objects
# loader = Loader(SAMPLE_RATE, OFFSET, WINDOW_DURATION, MONO)
padder = Padder()
log_spectrogram_extractor = LogSpectrogramExtractor(FRAME_SIZE, HOP_LENGTH)
feature_extractor = FeatureExtractor()
min_max_normaliser = MinMaxNormaliser(0, 1)
saver = Saver(SPECTROGRAMS_SAVE_DIR, MIN_MAX_VALUES_SAVE_DIR)
visualizer = Visualizer(SPECTROGRAMS_IMG_SAVE_DIR, FRAME_SIZE, HOP_LENGTH, OFFSET)
reconstructor = AudioReconstructor(RECONSTRUCTED_AUDIO_SAVE_DIR, HOP_LENGTH, FRAME_SIZE)

preprocessing_pipeline = PreprocessingPipeline()
# preprocessing_pipeline.loader = loader
preprocessing_pipeline.padder = padder
preprocessing_pipeline.extractor = log_spectrogram_extractor
preprocessing_pipeline.feature_extractor = feature_extractor
preprocessing_pipeline.normaliser = min_max_normaliser
preprocessing_pipeline.saver = saver
preprocessing_pipeline.reconstructor = reconstructor

number_of_clips = NUMBER_OF_CLIPS
for root, _, files in os.walk(AUDIO_FILES_DIR):
    for file in files:

        if re.match(AUDIO_REGEX, file) and number_of_clips > 0:
            print(file)
            number_of_clips -= 1
            y, sr = librosa.load(os.path.join(AUDIO_FILES_DIR, file), sr=SAMPLE_RATE, mono=MONO)
            duration = librosa.get_duration(y=y, sr=sr)
            window_id = 0
            while True:
                if (window_id) * WINDOW_DURATION >= duration:
                    break
                
                offset = window_id * WINDOW_DURATION
                loader = Loader(SAMPLE_RATE, offset, min(WINDOW_DURATION, duration - offset), WINDOW_DURATION, MONO)
                preprocessing_pipeline.loader = loader
                visualizer = Visualizer(SPECTROGRAMS_IMG_SAVE_DIR, FRAME_SIZE, HOP_LENGTH, offset)
                preprocessing_pipeline.visualizer = visualizer

                preprocessing_pipeline.process_file(AUDIO_FILES_DIR, file, window_id, save_flg=True, visualize_flg=False, external_features_flg=True)
                window_id += 1

00000-06 DI.wav
Loader: Shape of the loaded signal: (16317,)
Loader: Mean of the loaded signal: 0.0
Loader: Min of the loaded signal: 0.0
Loader: Max of the loaded signal: 0.0
Loader: Raw signal: [0. 0. 0. ... 0. 0. 0.]
FeatureExtractor: Amp ID: 0
FeatureExtractor: Clip ID: 0
LogSpectrogramExtractor: Shape of stft: (256, 64)
MinMaxNormaliser: Shape of norm_array: (256, 64)
MinMaxNormaliser: Max of norm_array: -100.0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:111: RuntimeWarning: invalid value encountered in true_divide


MinMaxNormaliser: Min of norm_array: -100.0
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
	00000-06-00 DI.wav
PreprocessingPipeline: Processed file 00000-06 DI.wav
Loader: Shape of the loaded signal: (16317,)
Loader: Mean of the loaded signal: 5.7258283e-05
Loader: Min of the loaded signal: -0.29922932
Loader: Max of the loaded signal: 0.16979657
Loader: Raw signal: [ 0.          0.          0.         ... -0.05721129 -0.05075676
 -0.04574516]
FeatureExtractor: Amp ID: 0
FeatureExtractor: Clip ID: 0
LogSpectrogramExtractor: Shape of stft: (256, 64)
MinMaxNormaliser: Shape of norm_array: (256, 64)
MinMaxNormaliser: Max of norm_array: 17.547225777674655
MinMaxNormaliser: Min of norm_array: -62.452774222325345
[[0.85590083 0.         0.         ... 0.392629   0.49172357 0.52878854]
 [0.85586812 0.         0.         ... 0.55046831 0.29483921 0.5